# Saving the values of period ratios from 1.1-1.5 from the NASA Exoplanet Archive to a csv
We want to save planets that are in trios that have an inner compact period ratio (1.1-1.5 since those below 1.1 should be unstable) to match our simulations for multiplanet (>3) systems to a csv to use for our other notebooks! A trio is a group of three adjacent planets in a system. The period ratio between the outer period ratios can be anything as long as the inner is compact.

In [3]:
#Import Statements & Define Constants
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import scipy
# import scipy.stats as stats
# from scipy.stats import lognorm
# from scipy.stats import ks_2samp
# import math
# import rebound
# import random
# from numpy.random import seed, random
# from scipy.stats import rayleigh
# from scipy.stats import norm
# import itertools
# from spock import FeatureClassifier
# from decimal import Decimal
# import pandas as pd
from tqdm import tqdm
# from matplotlib.ticker import EngFormatter
# from scipy.stats import gaussian_kde
# import matplotlib.pyplot as plt
# import seaborn as sns


# %matplotlib inline

# fmodel = FeatureClassifier()
# earth_mass = 5.97219*10**24
# sun_mass = 1.981 * 10**30
# mass_ratio = earth_mass/sun_mass

In [4]:
#MAKE INITIAL DATA FRAME

df = pd.read_csv('20230302_all_exoplanets.csv')

# Create an array of all host star/system names
hosts = df['hostname'].unique()

# For each planetary system...
for host in tqdm(hosts):
    # If the system contains less than 3 planets...
    if (df['hostname'] == host).sum() < 3:
        # Get the row indices of the system's planets
        index = df[df['hostname']==host].index
        # Remove those rows from our DataFrame
        df.drop(index, inplace=True)

# Data that might be useful to us
planet_data = pd.DataFrame()
planet_data["Planet Name"] = df['pl_name']
planet_data["Host Name"] = df['hostname']
planet_data["Orbital Period"] = df['pl_orbper']
planet_data["Discovery Method"] = df['discoverymethod']
planet_data["Semi-Major Axis (AU)"] = df['pl_orbsmax']
planet_data["Eccentricity"] = df['pl_orbeccen']
planet_data["Radius (R_earth)"] = df['pl_rade']
planet_data["Mass or Mass * sin(i) (M_earth)"] = df['pl_bmasse']
planet_data["Stellar Mass (M_sun)"] = df['st_mass']
planet_data.to_csv('planet_data.csv')
df = pd.read_csv('planet_data.csv', index_col=0)
df.head()

100%|█████████████████████████████████████████████████████| 3943/3943 [00:08<00:00, 482.92it/s]


,Planet Name,Host Name,Orbital Period,Discovery Method,Semi-Major Axis (AU),Eccentricity,Radius (R_earth),Mass or Mass * sin(i) (M_earth),Stellar Mass (M_sun)
17,2MASS J19383260+4603591 b,2MASS J19383260+4603591,406.0000,Eclipse Timing Variations,0.9200,0.330,13.4,603.8770,0.48
22,47 UMa b,47 UMa,1078.0000,Radial Velocity,2.1000,0.032,13.2,804.0800,1.06
23,47 UMa c,47 UMa,2391.0000,Radial Velocity,3.6000,0.098,14.2,171.6210,1.06
24,47 UMa d,47 UMa,14002.0000,Radial Velocity,11.6000,0.160,13.5,521.2200,1.06
27,55 Cnc b,55 Cnc,14.6516,Radial Velocity,0.1134,0.000,13.9,263.9785,0.91


In [5]:
df = pd.read_csv('planet_data.csv', index_col=0)
len(df)

1041

In [6]:
# GET RID OF ONES THAT DON'T HAVE INNER PRATIO 1.1-1.5
df['Keep'] = False

# for every system in planet_data
for system in df['Host Name'].unique():
    # new system df of JUST planets from that system
    system_df = df[df['Host Name'] == system]
    # sort them in order of orbital period lowest to highest
    system_sorted = system_df.sort_values(by='Orbital Period')
    
    Nplanets = system_df.shape[0] # number of planets in system
    Ntrios = Nplanets - 2 # number of trios, three adjacent planet groupings
    
    # for every trio
    for q in range(Ntrios):
        P1 = system_sorted.iloc[q]['Orbital Period']
        P2 = system_sorted.iloc[q+1]['Orbital Period']
        P3 = system_sorted.iloc[q+2]['Orbital Period']
        
        # if the inner period ratio is 1.1-1.5
        if P2/P1 > 1.1 and P2/P1 < 1.5:
            # keep these planets in dataframe
            df.loc[system_sorted.iloc[q].name, 'Keep'] = True
            df.loc[system_sorted.iloc[q+1].name, 'Keep'] = True
            df.loc[system_sorted.iloc[q+2].name, 'Keep'] = True
            

In [7]:
# create new data frame with trios that have the inner pratio 1.1-1.5
compact_df = df[df['Keep'] == True]

# get rid of 'keep' column since these would all be true and it's not important anymore
del compact_df['Keep']

print(len(compact_df))

compact_df.to_csv('compact_planet_data.csv')

df = pd.read_csv('compact_planet_data.csv', index_col=0)
df.head()

161


,Planet Name,Host Name,Orbital Period,Discovery Method,Semi-Major Axis (AU),Eccentricity,Radius (R_earth),Mass or Mass * sin(i) (M_earth),Stellar Mass (M_sun)
119,DMPP-1 b,DMPP-1,18.570,Radial Velocity,0.1462,0.083,5.29,24.27,1.21
120,DMPP-1 c,DMPP-1,6.584,Radial Velocity,0.0733,0.057,3.06,9.60,1.21
122,DMPP-1 e,DMPP-1,5.516,Radial Velocity,0.0651,0.070,1.86,4.13,1.21
194,GJ 180 b,GJ 180,17.133,Radial Velocity,0.0920,0.070,2.43,6.49,0.43
195,GJ 180 c,GJ 180,24.329,Radial Velocity,0.1290,0.090,2.41,6.40,0.43


In [5]:
# save only compact pratios 
pratios_observed = []

# for every system in the dataframe with only planets with inner pratio 1.1-1.5
for system in df['Host Name'].unique():
    # make new df of just that system's planets
    system_df = df[df['Host Name'] == system]
    # sort least to greatest orbital period
    system_sorted = system_df.sort_values(by='Orbital Period')
    Nplanets = system_df.shape[0] # number of planets in system
    
    # for every planet pair
    for z in range(Nplanets-1):
        # find periods
        P1 = system_sorted.iloc[z]['Orbital Period']
        P2 = system_sorted.iloc[z+1]['Orbital Period']
        # if the pratio is compact, add it to array
        if P2/P1 > 1.1 and P2/P1 < 1.5:
            pratios_observed.append(P2/P1)

# set up for dispersion

In [294]:
plist = [] # master plist, where each entry will be [P1,P2,..., PN] for each sim
for system in df['Host Name'].unique():
    periods = df[df['Host Name']==system]['Orbital Period'].values
    periods.sort()
    plist.append(periods)
plist
# populate trios with get_trios from rori code, drop the ones where the inner is not less than 1.5

[array([ 5.516,  6.584, 18.57 ]),
 array([ 17.133,  24.329, 106.3  ]),
 array([28.14 , 39.026, 62.24 ]),
 array([14.175685, 19.590025, 29.54115 ]),
 array([ 28.579743,  38.353037, 101.12    ]),
 array([ 5.75999,  7.28243, 10.86499, 25.1967 ]),
 array([12.1621839, 17.667087 , 29.79749  ]),
 array([ 5.24 ,  7.775, 10.115]),
 array([4.528598, 6.131243, 9.327527]),
 array([ 7.138048, 10.45582 , 14.76289 ]),
 array([ 4.44117,  6.42904, 14.09189]),
 array([ 6.679582,  9.715043, 13.62749 ]),
 array([ 5.577212,  7.760178, 15.189034]),
 array([0.2197 , 0.32528, 0.81161]),
 array([  7.008151,   8.719375,  14.44912 ,  91.93913 , 124.9144  ,
        210.60697 ]),
 array([ 5.28696,  7.07142, 10.3117 , 16.1457 ]),
 array([ 10.3039,  13.0241,  22.6845,  31.9996,  46.6888, 118.3807]),
 array([3.26663995, 4.27225018, 5.45298175]),
 array([10.3134, 13.7815, 23.0923]),
 array([2.89223021, 3.95116882, 5.10115756, 5.99273738]),
 array([ 6.195469,  8.348125, 13.767102]),
 array([ 7.466623, 11.131786, 16.259

In [295]:
# rori function
def p_ratios(P=None, logP=None, sim=None, log=True):
    """returns either the log or the normal period ratios of a system
       Param: a list of periods, log a, a, or a rebound sim
    """
    if logP != None:
        logpr = [logP[i+1]-logP[i] for i in range(len(logP)-1)]
        if log:
            return logpr
        else:
            return [10**x for x in logpr]
    if sim != None:
        ps = sim.particles[1:]
        P = [ps[i].P for i in range(len(ps))]   

    if log:
        return [np.log10(P[i+1]/P[i]) for i in range(len(P)-1)]
    else:
        return [P[i+1]/P[i] for i in range(len(P)-1)]
    
def get_trios(Plist=None, P=None, filter4=False, stable=None):
    """returns a tuple containing all the trios of adjacent planets 
        in population (lol of periods) of systems.
       optional param: stable: a list of booleans corresponding to
        the stability of each system. Returns the corresponding stability 
        of each trio
    """
    trios = []
    s = []
    
    if P!=None:
        Plist = [P]
        
    if len(Plist)==0:
        return []
    
    for i in range(len(Plist)):
        P = Plist[i]
        pr = p_ratios(P=P, log=False)
        for j in range(0, len(pr)-1):
            if P[j+1]/P[j]<1.5 and P[j+1]/P[j]>1.1:
                trios += [P[j:j+3]]
                if stable != None:
                    s += [stable[i]]

    if stable!=None:
        return s
    return trios


In [296]:
trios = get_trios(Plist = plist)
len(trios)

58

## Dispersion

$\sigma= \sqrt{\frac{1}{2}\sum_{j=1}^{2}{{(s_{j-1, j}-\overline{s})^2}}}$

Dispersion of the ith trio
$D_i = \frac{\sigma_{i}}{\overline{s}_i}$

Average dispersion $D = \sqrt{\frac{1}{N_{\mathrm{trios}}}\sum_{i=1}^{N_{\mathrm{trios}}}D_i^2}$

sj is log of period ratio

In [297]:
import pandas as pd
from matplotlib.ticker import ScalarFormatter, NullFormatter
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
from scipy.stats import pearsonr
from scipy.stats import linregress
import rebound
from IPython.display import display, clear_output

from spock import FeatureClassifier
model = FeatureClassifier()


#from ipynb.fs.full.functions import *

[02:52:59] WARNING: /croot/xgboost-split_1675457761144/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.


In [463]:
def dispersion(trios=None, systems=None, filter4=False, ddof=1): # input systems: orbital periods
    """returns the dispersion and error of a list of 
       trios as a tuple.
       For a single trio, only returns the dispersion
    """
    if systems != None:
        trios = get_trios(systems, filter4=filter4)
    
    if len(trios)==0:
        return 0, 0
    
    rel_var = []
    for t in trios:
        logpr = p_ratios(P=t, log=True)
        mu = np.mean(logpr)
        rel_var += [np.var(logpr)/mu**2]
    D = np.sqrt(np.mean(rel_var))
    if len(trios)>1: 
        error = np.std(rel_var, ddof=ddof)/np.sqrt(len(trios))#/(2*D)
    else:
        error = 10
    return D, error

In [517]:
trios = get_trios(Plist = plist)
array = []
for i in range(58):
    array.append(dispersion(trios=[trios[i]])[0])


In [524]:
import statistics
print(statistics.mean(array))
statistics.stdev(array)

0.24055875772573934


0.18492381874844405

In [464]:
stdev = dispersion(trios=get_trios(Plist = plist))
stdev

(0.3024495566800291, 0.01800678803123943)

In [ ]:
# check that the lengths of compact_df and these trios match, they do :)
trios = get_trios(Plist = plist)
flattened_list = [item for sublist in trios for item in sublist]

# Find unique values
unique_values = set(flattened_list)

# Get the count of unique values
count_unique_values = len(unique_values)

print(f"There are {count_unique_values} unique values")

# (0.302449556680029, 0.0180067880312394)

In [497]:
# def simulate_trios(D):
#     p1period = 1
#     p2period = np.random.uniform(1.1,1.5)
#     p3period = 0
#     while p3period < p2period:
#         s12= math.log10(p2period/p1period) 
#         s23= np.random.normal(np.log10(1.3),D*np.log10(1.3))
#         print(s12, D*s12, s23)
#         p3period = p2period*10**s23
#     return [p1period,p2period,p3period]


# trioslist=[simulate_trios(.302449556680029) for i in range(0,1000)]

In [337]:
np.random.normal(.11,.3*.11)

0.08584841129065729

In [338]:
dispersion(trios=trioslist)

(0.24428129211852947, 0.005066601607689658)

# done dispersion

In [350]:
# all period ratios that have inner 1.1 - 1.5
pratios_observed = np.asarray(pratios_observed)
pratios_observed.sort()
observed_period_cdf = np.cumsum(pratios_observed)/pratios_observed.sum()

In [352]:
len(pratios_observed)

71